In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
zomato=pd.read_csv("/kaggle/input/zomato-bangalore-restaurants/zomato.csv")
zomato.head()

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
zomato.isna().sum()

In [ ]:
zomato=zomato.drop(columns=(["url","phone","reviews_list"]))
zomato.head()

In [ ]:
zomato.drop_duplicates(inplace=True)

In [ ]:
zomato.isna().sum()

In [ ]:
zomato.info()

In [ ]:
zomato.dropna(how="any",inplace=True)
zomato.isna().sum()

In [ ]:
zomato.columns

In [ ]:
zomato=zomato.rename(columns={"approx_cost(for two people)":"cost","listed_in(type)":"type","listed_in(city)":"city"})

In [ ]:
zomato.head()

In [ ]:
#Removing '/5' from Rates
zomato = zomato.loc[zomato.rate !='NEW']
zomato = zomato.loc[zomato.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')
zomato['rate'].head() # looking at the dataset after transformation

In [ ]:
zomato["rate"].unique()

In [ ]:
# Adjust the column names
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(1, 0),inplace=True)
zomato.book_table.replace(('Yes','No'),(1, 0),inplace=True)
zomato.head() # looking at the dataset after transformation

In [ ]:
#Encode the input Variables
def Encode(zomato):
    for column in zomato.columns[~zomato.columns.isin(['rate', 'cost', 'votes', 'online_order', 'book_table'])]:
        zomato[column] = zomato[column].factorize()[0]
    return zomato

zomato_en = Encode(zomato.copy())
zomato_en.head() # looking at the dataset after transformation

In [ ]:
zomato_en["cost"].unique()

In [ ]:
corr = zomato_en.corr(method='kendall')
plt.figure(figsize=(15,8))
sns.heatmap(corr, annot=True)

### There is no much correlation between any features

# Data Visualization

In [ ]:
sns.countplot(zomato['online_order'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Which type of delivery people prefer more')

### It can be seen that people in banglore prefers online orders more

In [ ]:
sns.countplot(zomato['book_table'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Restaurants table book counts')

### We can see that as the online orders are the table book counts is very less

## Note:
###     So if some business people are looking to start a new restaurant they must surely tie up with some online delivery partners to make more sales.

In [ ]:
sns.countplot(zomato['rate'])
fig = plt.gcf()
fig.set_size_inches(20,20)
plt.title('Restaurants rating visualization')

### From the above graph we can say that the good quality and rated restaurants are more.
### So the competition is absolutely very high

In [ ]:
fig=plt.gcf()
fig.set_size_inches(10,10)
sns.stripplot(x=zomato["online_order"],y=zomato["rate"])
plt.title("Which type of customer giving rating")

### Almost they are equally rated

In [ ]:
fig=plt.gcf()
fig.set_size_inches(10,10)
sns.stripplot(x=zomato["book_table"],y=zomato["rate"])
plt.title("book_tabel Vs rate")

### here we can see that the restaurants which have table booking is mostly rated above 3

In [ ]:
sns.countplot(zomato['city']).set_xticklabels(sns.countplot(zomato['city']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(13,13)
plt.title('Location wise count for restaurants')
plt.show()

### We can see there is less no of restaurants in electronic city and new BEL road

In [ ]:
z=zomato[zomato["city"]=="Electronic City"]

In [ ]:
b=zomato[zomato["city"]=="New BEL Road"]

In [ ]:
c=[z,b]

In [ ]:
plt.figure(figsize=(15,10), facecolor='w')
plotnumber = 1

for column in c:
    if plotnumber<=4 :
        ax = plt.subplot(2,2,plotnumber)
        sns.stripplot(x=column["city"],y=zomato["rate"])
        #plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

In [ ]:
sns.countplot(zomato['rest_type'])
sns.countplot(zomato['rest_type']).set_xticklabels(sns.countplot(zomato['rest_type']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(15,15)
plt.title('Restuarant Type')

In [ ]:
plt.figure(figsize=(15,10), facecolor='w')
plotnumber = 1

for column in c:
    if plotnumber<=4 :
        ax = plt.subplot(2,2,plotnumber)
        sns.countplot(column['rest_type']).set_xticklabels(sns.countplot(column['rest_type']).get_xticklabels(), rotation=90, ha="right")
        plt.title(column["city"].unique()[0])
    plotnumber+=1
plt.show()

In [ ]:
plt.figure(figsize=(15,10), facecolor='w')
plotnumber = 1

for column in c:
    if plotnumber<=2 :
        ax = plt.subplot(2,1,plotnumber)
        sns.countplot(column['cost']).set_xticklabels(sns.countplot(column['cost']).get_xticklabels(), rotation=90, ha="right")
        plt.title(column["city"].unique()[0])
    plotnumber+=1
plt.show()

### In these two areas the restaurants are less in number and the more available types is of casual dining and quick bites 
### So if we start other types of restaurants in these areas there will not be a huge competition
### And average cost for 2 people in these areas are between 400-700 rupees
### So if we start a restaurant and sell the foods at cheaper rates , the restaurants can sustain well in these areas

In [ ]:
type_plt=pd.crosstab(zomato['rate'],zomato['type'])
type_plt.plot(kind='bar',stacked=True);
plt.title('Type - Rating',fontsize=15,fontweight='bold')
plt.ylabel('Type',fontsize=10,fontweight='bold')
plt.xlabel('Rating',fontsize=10,fontweight='bold')
plt.xticks(fontsize=10,fontweight='bold')
plt.yticks(fontsize=10,fontweight='bold');

In [ ]:
sns.countplot(zomato['cost'])
sns.countplot(zomato['cost']).set_xticklabels(sns.countplot(zomato['cost']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(20,20)
plt.title('Cost of Restuarant')

### More people in bangalore opt for restaurants only if the average cost for 2 people is within 1,000 rupees

In [ ]:
plt.figure(figsize=(15,7))
chains=zomato['name'].value_counts()[:20]
sns.barplot(x=chains,y=chains.index,palette='Set1')
plt.title("Most famous restaurant chains in Bangaluru",size=20,pad=20)
plt.xlabel("Number of outlets",size=15)

### Onesta is more famous in Bangalore

In [ ]:
#Some Transformations
zomato_en['cost'] = zomato_en['cost'].astype(str) #Changing the cost to string
zomato_en['cost'] = zomato_en['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato_en['cost'] = zomato_en['cost'].astype(float) # Changing the cost to Float
zomato_en.info() # looking at the dataset information after transformation

# Lets split the data for test and train

In [ ]:
x=zomato_en.drop(columns=(["rate"]))
y=zomato_en["rate"]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=420)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
ada=AdaBoostRegressor(n_estimators=90)
lin=LinearRegression()
ran=RandomForestRegressor(n_estimators=100)
dtc=DecisionTreeRegressor(random_state=420)
etr=ExtraTreesRegressor(n_estimators=100)
gb=GradientBoostingRegressor(n_estimators=100)


In [ ]:
models = {"Linear Regression" : lin,
          "Random forest" : ran,
          "AdaBoost" : ada,"Gradient Boost":gb,"Decision Tree":dtc,"Extra Tree":etr}
scores= { }
Train_set_scores={}

In [ ]:
for key, value in models.items():    
    model = value
    model.fit(x_train, y_train)
    scores[key] = model.score(x_test, y_test)
    Train_set_scores[key]=model.score(x_train,y_train)

In [ ]:
datal = {"train":Train_set_scores,"test":scores} 
  
# Creates pandas DataFrame. 
dfl = pd.DataFrame(datal, index =scores.keys())
dfl.sort_values(by=["train","test"], axis=0 ,ascending=False, inplace=True) 
# print the data 
dfl

# Here it can observed that we got best accuracy for Extra Tree Regressor